In [17]:
import gensim
from gensim import corpora
from gensim import similarities
from gensim import models
from gensim.models import CoherenceModel

import preprocess

# Load LDA Model

In [18]:
lda_disk=gensim.models.ldamodel.LdaModel.load("Model/finalmodel_5Topics")

In [19]:
id2word = corpora.Dictionary.load('Model/finalmodel_Dictionary')

# Extract & Preprocess Test Data

In [20]:
test_corpus = preprocess.load_corpus('Data/Test/Chapters/65145')

test_ids = test_corpus.fileids()
chapters_name = [id.replace('.txt','') for id in test_ids]

test_docs = preprocess.corpus2docs(test_corpus)

In [21]:
bigram = gensim.models.Phrases(test_docs, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[test_docs], threshold=50)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

docs_bigrams = preprocess.make_bigrams(bigram_mod, test_docs)
data_bigrams_trigrams = preprocess.make_trigrams(bigram_mod, trigram_mod, docs_bigrams)

In [22]:
test_vecs = preprocess.docs2vecs(data_bigrams_trigrams, id2word)


In [23]:
for i in range(0,len(test_vecs)):
    vector = lda_disk[test_vecs[i]]
    sim_topic = max(vector,key=lambda item:item[1])
    print("test doc" + str(i) + ": " + str(vector))
    print("Closest Topic: Topic " + str(sim_topic[0]))

test doc0: [(0, 0.18936332), (1, 0.19214728), (2, 0.4796578), (3, 0.046316046), (4, 0.09251557)]
Closest Topic: Topic 2
test doc1: [(0, 0.21943791), (1, 0.19598575), (2, 0.4601552), (3, 0.054990157), (4, 0.06943099)]
Closest Topic: Topic 2
test doc2: [(0, 0.071723826), (1, 0.12538408), (2, 0.6191744), (3, 0.036312893), (4, 0.14740476)]
Closest Topic: Topic 2
test doc3: [(0, 0.120798655), (1, 0.09923054), (2, 0.46599904), (3, 0.026383476), (4, 0.28758833)]
Closest Topic: Topic 2
test doc4: [(0, 0.11447889), (1, 0.1094327), (2, 0.63737774), (3, 0.031577718), (4, 0.10713295)]
Closest Topic: Topic 2
test doc5: [(0, 0.17712101), (1, 0.16861315), (2, 0.5106882), (3, 0.028049937), (4, 0.115527704)]
Closest Topic: Topic 2


In [24]:
topic_word = lda_disk.show_topic(0, topn=len(id2word))
topic_word[0:20]

[('conception', 0.032495115),
 ('zarathustra', 0.024165511),
 ('thou', 0.021026777),
 ('hath', 0.015878359),
 ('pure', 0.014599969),
 ('phenomenon', 0.013353135),
 ('thee', 0.013118728),
 ('unto', 0.012600138),
 ('cognition', 0.011412212),
 ('intuition', 0.011389661),
 ('empirical', 0.00927787),
 ('social', 0.009052046),
 ('education', 0.009046041),
 ('activity', 0.009034674),
 ('representation', 0.008575148),
 ('priori', 0.0080437185),
 ('verily', 0.007781609),
 ('doth', 0.00770331),
 ('space', 0.007673117),
 ('transcendental', 0.0074760346)]

## Model 1 - Using Similarity Score

### Top Keywords

In [25]:
chap_num = 0
vector = lda_disk[test_vecs[chap_num]]
sim_topic = max(vector,key=lambda item:item[1])
top_topic = sim_topic[0]
top_topic
topic_word = lda_disk.show_topic(top_topic, topn=len(id2word))
# topic_word

In [26]:
selected_words = [id2word[i[0]] for i in test_vecs[chap_num]]
selected_words[0:10]

['able',
 'absolute',
 'absolutely',
 'accept',
 'accessory',
 'accompany',
 'accordingly',
 'account',
 'achievement',
 'acknowledge']

In [27]:
key_words = []

keyword_type = 'bigrams'
for word in topic_word:
    if(keyword_type == 'unigrams'):
        if (len(key_words) < 5) & (word [0] in selected_words):
            key_words.append(word)
    else:
        if ('_' in word[0]) & (len(key_words) < 5) & (word [0] in selected_words):
            key_words.append(word)

key_words

[('outer_world', 0.00080459705),
 ('natural_selection', 9.627251e-06),
 ('reproductive_organ', 6.3105076e-06),
 ('male_female', 6.2402873e-06),
 ('internal_secretion', 6.216118e-06)]

### Recommender

In [28]:
chapters_name

['I', 'II', 'III', 'IV', 'V', 'VI']

In [41]:
chap_num = 4

chosen_chapter = chapters_name[chap_num]
recommendation_scores = []
similarity = similarities.MatrixSimilarity(lda_disk[test_vecs])

for i in range(0,len(test_vecs)):
    vector = lda_disk[test_vecs[i]]
    sim_topic = max(vector,key=lambda item:item[1])
    
    if(i == chap_num):
        sims = similarity[vector]
        sims = list(enumerate(sims))
        for sim in sims:
            chapter_num = sim[0]
            recommendation_score = [chapters_name[chapter_num], sim[1]]
            recommendation_scores.append(recommendation_score)
        
recommendation_scores = sorted(recommendation_scores, key=lambda x: x[1], reverse=True)     
recommendation = []


print('Top 3 Recommended Chapters:')
for i in range(1,4):
    recommendation.append((recommendation_scores[i][0], recommendation_scores[i][1]))
    print(f'{i}. {str(recommendation_scores[i][0])}({str(recommendation_scores[i][1])})')

Top 3 Recommended Chapters:
1. III(0.9955204)
2. VI(0.9793973)
3. I(0.964519)


## Model 2 - Using Distribution of Topics

In [30]:
chapters_name

['I', 'II', 'III', 'IV', 'V', 'VI']

In [42]:
# if first digit in tuple matches, store doc
# reco_docs -- dont remove user input
# - get index of user input within reco_docs
# - compare difference between first topic probability and store as list; e.g. {4: 0.002,...}
# - take top 5 minimum difference

# chap_num = 0
vector_selected = lda_disk[test_vecs[chap_num]]
vector_selected.sort(key=lambda x: x[1], reverse=True)
vector_selected

reco_docs = {}

for i in range(0,len(test_vecs)):
    vector = lda_disk[test_vecs[i]]
    vector.sort(key=lambda x: x[1], reverse=True)
    
    if (vector[0][0]==vector_selected[0][0]) & (vector[1][0] == vector_selected[1][0]):
        reco_docs[i] = vector
        # print(vector[0][1])

# print(reco_docs)
input_topic = reco_docs[chap_num]

# Remove user input
if chap_num in reco_docs.keys():
    reco_docs.pop(chap_num)

# if there's no 2 same topics
if len(reco_docs) < 3:
    for i in range(0,len(test_vecs)):
        vector = lda_disk[test_vecs[i]]
        vector.sort(key=lambda x: x[1], reverse=True)
        
        if (vector[0][0]==vector_selected[0][0]):
            reco_docs[i] = vector

    # Remove user input
    if chap_num in reco_docs.keys():
        reco_docs.pop(chap_num)

diff = ""
diff_dict = {}
for x, y in reco_docs.items():
    diff = abs(input_topic[0][1] - reco_docs[x][0][1])
    diff_dict[x] = diff

diff_dict = sorted(diff_dict.items(), key=lambda x:x[1], reverse=False)
diff_dict = dict(diff_dict)
# print(diff_dict)

lst = list(diff_dict.keys())
# print(lst[0:5])

print('Top 3 Recommended Chapters:')
for i in range(0,len(lst)):
    if i < 3:
        print(f'{i+1}. {chapters_name[lst[i]]}')

Top 3 Recommended Chapters:
1. III
2. VI
3. I
